# Sleep spindles detection

## Importations

Commençons par importer les librairies utilisées.

In [8]:
import numpy as np 
import scipy.signal as sg
import matplotlib.pyplot as plt
import h5py
import importlib
import myAlgos

Importons maintenant les données EEG.

In [26]:
importlib.reload(myAlgos)
data_spindles = h5py.File('data_spindles.h5')    
n_days = len(data_spindles) # n=7
eeg_signals = [None] * n_days
hypnograms = [None] * n_days
hypnograms_long = [None] * n_days

for i in range(n_days):
    path = 'day_' + str(i) + '/'
    eeg_signals[i] = data_spindles[path + 'eeg_signal']
    hypnograms[i] = data_spindles[path + 'hypnogram']

# Cela permet de recuperer un stade de sommeil facilement :
for i in range(n_days):
    hypnograms_long[i] = np.empty(len(eeg_signals[i]))
    hypnograms_long[i][:] = np.nan
    len_cast = int(len(eeg_signals[i])/len(hypnograms[i]))
    for j in range(len(hypnograms[i])):
        hypnograms_long[i][j*len_cast:(j+1)*len_cast] = hypnograms[i][j]
    hypnograms_long[i][np.where(np.isnan(hypnograms_long[i]))] = 0
# Ainsi, on peut obtenir simplement : stade de sommeil du point eeg_signals[i][124] <=> hypnograms_long[124] 

for i in range(n_days):
    l = len(eeg_signals[i])
    t = myAlgos.pointsToTime(l)
    print("jour %s : %s points | %sh %sm %ss" %(i, l, t[0], t[1], t[2]))

jour 0 : 7329225 points | 8h 8m 36s
jour 1 : 7785250 points | 8h 39m 1s
jour 2 : 6761103 points | 7h 30m 44s
jour 3 : 7024938 points | 7h 48m 19s
jour 4 : 7031288 points | 7h 48m 45s
jour 5 : 7744777 points | 8h 36m 19s
jour 6 : 7913879 points | 8h 47m 35s


Le signal EEG de chaque jour est accessible par :  ```eeg_signals[jour]```

L'hypnogramme d'un point en particulier peut être obtenu par : ```hypnograms_long[jour][point]```.

## Filtre passe-bande

Filtrons le signal entre 11Hz et 15Hz avec un passe-bande Butterworth d'ordre 5.

In [10]:
Fs = 250
b,a = sg.butter(5,(11/Fs, 15/Fs),'bandpass')
eeg_signals_filt = [None] * n_days
for i in range(n_days):
    eeg_signals_filt[i] = sg.filtfilt(b,a,eeg_signals[i])

C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


## Organisation du signal en epochs

Divisons le signal en epochs de 5 secondes :

In [11]:
n_pts_epochs = 1250
# Il faut rendre le nombre de pts du signal divisible par la taille d'un epoch
for i in range(n_days):
    reste = len(eeg_signals[i])%n_pts_epochs
    eeg_signals[i] = eeg_signals[i][reste:]
    eeg_signals_filt[i] = eeg_signals_filt[i][reste:]
    hypnograms_long[i] = hypnograms_long[i][reste:]
    
def signal_to_epochs(eeg_data):
    epochs = [None] * n_days
    # n epochs, 0.5s/epoch --> (n, 125)
    for i in range(n_days):
        n_epochs = int(len(eeg_data[i])/n_pts_epochs)
        epochs[i] = np.reshape(eeg_data[i], (n_epochs, n_pts_epochs))
    return epochs

epochs = signal_to_epochs(eeg_signals)
epochs_filt = signal_to_epochs(eeg_signals_filt)
hypno_epochs = signal_to_epochs(hypnograms_long)

Exemple pour acceder à l'epoch 10 du jour 2 : ```epochs[2][10]```.

On peut récupérer l'hypnograme d'un epoch par : ```hypno_epoch[jour][epoch]```.

In [13]:
eeg_signals_filt_nan = [None] * n_days
for i in range(7):
    #Enlever les valeurs excessivement grandes
    amp_excess = np.where(np.abs(eeg_signals_filt[i]) > 80)[0]
    eeg_signals_filt_nan[i] = np.copy(eeg_signals_filt[i])
    eeg_signals_filt_nan[i][amp_excess] = np.NaN

epochs_filt_nan = signal_to_epochs(eeg_signals_filt_nan)

#Detection des spindles
spindles_detected = 0
spindles_positions = [None] * n_days
spindles_heights = [None] * n_days
spindles_length = [None] * n_days

fails = np.array([0,0,0,0,0,0])
for i in range(n_days): #Parcours par jour
    spindles_positions[i] = []
    spindles_heights[i] = []
    spindles_length[i] = []
    for j in range(epochs_filt_nan[i].shape[0]): #Parcours par epoch
        if j == 0: continue #petit probleme au tout debut a cause d'artefacts
        current_epoch = epochs_filt_nan[i][j]
        if (not myAlgos.nanFound(current_epoch)) and \
            myAlgos.threshold_reached(current_epoch) :
            
            # On se centre autour de la valeur max.
            # 600 points <=> 2.4 secondes
            max_pos = np.argmax(current_epoch)
            max_val = np.max(current_epoch)
            window = eeg_signals_filt_nan[i][j*n_pts_epochs + max_pos - 300 : j*n_pts_epochs + max_pos + 300]
            if not myAlgos.nanFound(window):    
                peaks, peak_properties = sg.find_peaks(window, height=0)
                peak_heights = peak_properties['peak_heights']
                if max_val == np.max(peak_heights) : #si rien de bizare lors du fenetrage
                    wave_peaks, wave_heights = myAlgos.keepWavePeaks(peaks, peak_heights)
                    if not myAlgos.isTooShort(wave_peaks): #si >0.5s
                        if myAlgos.isSymmetric(wave_heights): #si les cretes sont symmetriques
                            if myAlgos.isTooHigh(wave_heights): #Si l'amplitude min n'est pas trop grande
                                spindles_detected += 1
                                spindles_positions[i].append(j*n_pts_epochs + max_pos)
                                spindles_heights[i].append(np.round(np.max(wave_heights),2))
                                spindles_length[i].append(myAlgos.waveLength(wave_peaks))
                            else :
                                fails[5] += 1
                                continue
                        else :
                            fails[4] += 1
                            continue
                    else :
                        fails[3] += 1
                        continue
                else : 
                    fails[2] += 1
                    continue
            else :
                fails[1] += 1
                continue
        else : 
            fails[0] += 1
            continue 
print('spindles détectées : ', spindles_detected)

spindles détectées :  4497
